# Before We Begin

You will need:
  1. a virtualenv
  2. Jupyter notebook running a kernel in that virtualenv
  3. Klein and Treq (and therefore Twisted) installed in that kernel

To get those:

```console
$ virtualenv abc
$ . abc/bin/activate
$ pip install jupyter[notebook] klein treq
$ jupyter notebook Twisted-Tutorial.ipynb
```

The first thing we need to do is to set up the klein application. If what follows seems a bit arcane, there is no need to worry: it is only needed to integrate Klein properly with Jupyter. Every way of deploying a Klein application will need, eventually, some sort of "rubber meets road" part, where we connect the application to the network. There will always be some complicating consideration: which low-level event loop to use? What kind of sockets to listen on? Here is the specific way we do it for this demo on Jupyter.

Another compromise done here is that the entire initialization is in one cell. From a pedagogical perspective, it would have been better to split it. However, this way it is much easier to hit "restart kernel" and then re-run the initialization. 

So what does happen here? (It is safe to skip this part) Jupyter is running on top of the Tornado event loop. Twisted can run its "reactor", the Twisted event loop, on top of Tornado's. This is not the default -- the default is the native-Twisted event loop. The first thing we do is install the Tornado reactor. It is important this is the first thing to be done: Twisted will automatically install the default reactor as soon as a reactor is requested, if one is not already installed.

Then, we created a server endpoint which listens on port 8080. We get the global Klein resource, wrap it in a Site object and connect it to the endpoint. Note that for production use of Klein, it is best not to use the global resource. However, for a quick prototype, it is extremely useful.

In [1]:
from tornado.platform.twisted import install
reactor = install()

from twisted.web.server import Site
from twisted.internet import endpoints
from klein import route, resource

description = "tcp:8080"

ep = endpoints.serverFromString(reactor, description)
s = Site(resource())
ep.listen(Site(resource()))

<Deferred at 0x10b547518 current result: <<class 'twisted.internet.tcp.Port'> of twisted.web.server.Site on 8080>>

As is traditional, the first thing to be done in any programming environment is to greet. This allows both showing what is the "minimum program" as well as have an easy way to demonstrate the environment is working. Klein uses decorators to indicate routing. We put a resource at the root which greets with a constant string.

In [3]:
from klein import route
@route('/')
def home(request):
    return 'Hello, everybody!'

The excellent webbrowser module can be used to open a browser to a given page. If this is running locally (and not through a virtualization environment or a cloud environment), the following will open a browser to a pleasant (if generic) greeting from Klein.

In [4]:
import webbrowser
webbrowser.open("http://127.0.0.1:8080/")

True

Constant strings, though, are boring. The reason to use a web application framework is, well, to write applications. We personalize the greeting by using string interpolation. Maybe it is not the fanciest templating solution, but it will do for now.

In [5]:
@route('/greet/<user>')
def greet(request, user):
    return 'Hello {}!'.format(user.upper())

Try changing the URL below to have your name (or your nickname, or that of a friend that needs to be shown the power of Klein and Twisted).

In [6]:
webbrowser.open("http://localhost:8080/greet/your-name")

True

Up until now, the code would have been similar if a WSGI framework like Flask or Bottle. Returning a string, maybe interpolating some argument and doing some processing. Most web applications, however need to do something interesting -- and often it is something that takes time. A typical example is to get data from a web API. While in real-life, the web API will return something interesting, for now we will satisfy ourselves with the API version of whats-my-IP.

In [3]:
import json
from twisted.internet import defer
import treq

@route('/ip')
@defer.inlineCallbacks
def getip(request):
    url = 'https://httpbin.org/ip'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    ip = value['origin']
    defer.returnValue('From {}'.format(ip))

We use treq, an asynchronous library inspired by "requests". Notice that treq.get() cannot block, so it returns an unfinished result. In order to have to get the value, we need to "yield" it. The Twisted event loop will return to the function when the results are ready. Notice in this example we yield twice: once waiting for the headers, and once getting the body. Also notice that because this is a generator, it is invalid to return from it so we use a special function, "returnValue". In order to mark this as a function that needs to do asynchronous work with "inlineCallbacks". This decorator is called this because in essence, it is replacing explicit callbacks with "inline" callbacks: code that happens after the yield is effectively run as a separate callback.

In [6]:
import webbrowser
webbrowser.open("http://127.0.0.1:8080/ip")

True

In the real world sometimes services are slow. Often, to figure out exactly how slow a service is we need to delve into metrics collections and aggregations. For this example, we will slow a service down to human-noticable speeds: ten seconds. The "delay" service returns the same data, but with configurable delay. In order to make it even more painful, we will call it twice. This stands in for calling more than one backend service. As an example, imagine a shopping cart application needing to get the list of items in the cart from one service, and the "suggested items" from another in order to render a complete page.

In [7]:
@route('/compare')
@defer.inlineCallbacks
def compare(request):
    url = 'https://httpbin.org/delay/10'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    first_ip = value['origin']
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    second_ip = value['origin']    
    defer.returnValue('These are the same:{}=={}'.format(first_ip, second_ip))

This takes 20 seconds -- once to get the IP the first time and once to get it the second time. This code is not all that different from what we would write if we were using, say, Flask. The yields would not be there, and requests would be used instead of treq, but we could have a one-to-one transcription. That includes the latency, as well. Flask would wait 10 seconds for the first IP and 10 seconds for the second one for a total of 20 seconds. 

In [8]:
webbrowser.open("http://127.0.0.1:8080/compare")

True

The page cannot render in less than 10 seconds -- we need that much for even one response. But does it have to take more? We can send both requests at the same time, wait until they both come back and then return the result. This could be done in a classical WSGI framework: if we were willing to use a separate thread pool and use various thread-communication mechanisms. With Twisted, we just use deferreds directly. Deferreds represent a result that has not arrived yet. A deferred can have a callback attached which indicates what to do when the result arrived. Deferreds will call each callback with the return value of the previous one -- with one exception. If a callback returns a deferred, the next one will be called with the *result* of the deferred.

Last but not least, we use the function gatherResults, which takes a list of deferreds and returns a deferred that fires with a list of results. This allows us to send both requests in parallel, but to process the results only when both arrive. Again, keep in mind the shopping cart example to realize the usefulness.

In [8]:
@route('/faster-compare')
def faster_compare(request):
    url = 'https://httpbin.org/delay/10'
    d = treq.get(url)
    d.addCallback(lambda response: response.content())
    d.addCallback(json.loads)
    d.addCallback(lambda dct: dct['origin'])
    d2 = treq.get(url)
    d2.addCallback(lambda response: response.content())
    d2.addCallback(json.loads)
    d2.addCallback(lambda dct: dct['origin'])
    both = defer.gatherResults([d, d2])
    def do_format(ips):
        first_ip, second_ip = ips
        return 'These are the same:{}=={}'.format(first_ip, second_ip)
    both.addCallback(do_format)
    return both

There has been anecdotal results, if not a lot of published research, that reducing latency is a big boon in user retention on dynamic web pages. These techniques are useful to accomplish just that: reduce latency in a micro-service architecture. For example, notice how it is much less annoying to wait for the lower-latency version of the page.

In [9]:
webbrowser.open("http://127.0.0.1:8080/faster-compare")

True

Finding our IP several time is all well and good, but it gets boring after a while. We can make our own web service! With beer and ice cream! It will add numbers, and take two seconds to do so. We introduce, for this, another Twisted function will be used: deferLater. It takes a timeout, a function and arguments. After the timeout has lapsed, it will call the function with the arguments, and fire the deferred with the result.

Note that naively, in order to implement a slow API, some think to use sleep. However, in a Twisted program, we can never call sleep -- it would pause the entire event loop. deferLater is actually build on a primitive function, reactor.callLater, which is used in those cases where a time-delay is needed.

In [10]:
from twisted.internet import task
import operator

@route('/add/<int:a>/<int:b>')
def add(request, a, b):
    d = task.deferLater(reactor, 2, operator.add, a, b)
    d.addCallback(str)
    return d

Apparently, the canonical demonstration of addition is with 3 and 4.

In [11]:
webbrowser.open("http://127.0.0.1:8080/add/3/4")

True

We wrap the service in a function that looks local -- almost. It does return a deferred, since it can block for a long time (for example, if the addition service is overloaded). This is a typical pattern: we can often find high level wrappers to isolate the details of the remote protocol, but the essential fact that it is remote, and can be slow (and potentially fail) remains.

We then show the same ideas as above, in a different context: add4 and fast-add4 both are correct additions, but one finishes two seconds after the other.

In [16]:
def remote_add(x, y):
    url = 'http://localhost:8080/add'
    d = treq.get(url + '/{}/{}'.format(x, y))
    d.addCallback(lambda response: response.content())
    return d

@route('/add4/<int:a>/<int:b>/<int:c>/<int:d>')
@defer.inlineCallbacks
def add4(request, a, b, c, d):
    x = yield remote_add(a, b)
    y = yield remote_add(c, d)
    res = yield remote_add(x, y)
    defer.returnValue(str(res))

@route('/fast-add4/<int:a>/<int:b>/<int:c>/<int:d>')
def fast_add4(request, a, b, c, d):
    d = defer.gatherResults([remote_add(a, b), 
                             remote_add(c, d)])
    @d.addCallback
    def final_add(results):
        x, y = results
        return remote_add(x, y)
    return d

In [17]:
webbrowser.open("http://127.0.0.1:8080/add4/3/4/9/14")
webbrowser.open("http://127.0.0.1:8080/fast-add4/3/4/9/14")

True

Twisted being single threaded means shared data manipulation is straightforward. Below we show a simple counter example. This is not to recommend global integers, but this technique is useful, with proper abstraction. It can be used for in-process caching or for in-process statistics that are ok to lose if a process crashes.

In [18]:
counter = 0
@route('/count')
def count(request):
    global counter
    counter += 1
    return str(counter)

In [19]:
webbrowser.open("http://127.0.0.1:8080/count")
webbrowser.open("http://127.0.0.1:8080/count")
webbrowser.open("http://127.0.0.1:8080/count")

True

Twisted also has in-process time-based loops. This allows integrating periodic tasks in the same application, simplifying deployment. For example, this can be used for cache flush or clearing, for log rotation and other similar purposes.

In [20]:
from twisted.internet import task
loops = 0
def add_loop():
    global loops
    loops += 1

@route('/loops')
def show_loops(request):
    return str(loops)

lc = task.LoopingCall(add_loop)
d = lc.start(5)

In [21]:
reactor.callLater(6, webbrowser.open, "http://127.0.0.1:8080/loops")
reactor.callLater(12, webbrowser.open, "http://127.0.0.1:8080/loops")

Finally we show how to directly create, and fire, deferreds. This is, in essence, a minimal chat engine. The "/block" is a long poll, waiting for a message to come in. /unblock/ sends a message to the client that has been listening the longest. Note that because Twisted is asynchronous, "longest" is well defined.

In [25]:
blocking = []

@route('/block')
def count(request):
    result = defer.Deferred()
    blocking.append(result)
    result.addCallback(lambda x: x.upper())
    return result

@route('/unblock/<string:value>')
def unblock(request, value):
    blocking.pop(0).callback(value)
    return u"OK " + value

In [28]:
webbrowser.open("http://127.0.0.1:8080/block")

True

In [29]:
webbrowser.open("http://127.0.0.1:8080/unblock/hello%20universe")

True